In [2]:
import json
import os
import numpy as np
import math
from typing import Dict, List, Tuple
import copy

from datetime import timedelta
from config import *
from utils import *
from main import *
from GtConflictFinder import GtConflictFinder
from ConflictDetector import ConflictDetector

import plotly.graph_objects as go
from plotly.subplots import make_subplots
test_projects = [
    "HomeA/2016",
    "HomeB/2016",
    "HomeC/2016",
    "HomeD/2016",
    "HomeF/2016",
    "HomeG/2016",
]
ctx_info = ContextAccessor({
            TIME_CTX: {
                "range" : (0, 24*60),
                "interval" : 20,
            },
            # "humidity#NUM" : {
            #     "range" : (0., 1.0),
            #     "interval" : 0.1,
            # },
            # "apparentTemperature#NUM" : {
            #     "range" : (-10., 100),
            #     "interval" : 10,
            # },
            # "summary#CAT": {},
            WEEKDAY_CTX: {
                "range": (0, 6),
                "interval": 1,
            },
        })
capacity = {
    "Range" : 2,
    "Microwave": 1,
    "LivingLights": 0,
    "HomeOffice": 1,
    "WashingMachine": 1,
}


In [14]:
habit_groups = {}
for p in test_projects:
    habit_groups[p], _ = test_umass(test_project=p, ctx_info=ctx_info,train_ratio=0.7, ccp_alpha=10e-6)

DEBUG:root:The number of device events from processed file: {'Range': 377, 'Microwave': 931, 'LivingLights': 1099, 'HomeOffice': 997}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 6119, 'temperature#NUM': 6119, 'humidity#NUM': 5418, 'summary#CAT': 1835}
DEBUG:root:The number of device events from processed file: {'LivingLights': 1384, 'Microwave': 619}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 6112, 'temperature#NUM': 6118, 'humidity#NUM': 5422, 'summary#CAT': 1290}
DEBUG:root:The number of device events from processed file: {'HomeOffice': 746, 'Microwave': 1425, 'LivingLights': 1283}
DEBUG:root:The number of context events from processed file: {'apparentTemperature#NUM': 6115, 'temperature#NUM': 6112, 'humidity#NUM': 5404, 'summary#CAT': 1322}
DEBUG:root:The number of device events from processed file: {'Microwave': 929, 'LivingLights': 831, 'WashingMachine': 215, 'Range': 871}
DEBUG:root:The num

In [15]:
#Exps:
#   1. Contrived environment with injected conflicts:
#   2. Real dataset with groundtruth conflicts:
#   3. observations about the algorithm: with different threshold, different capacity for the devices, different number of users.

In [16]:
for home, groups in habit_groups.items():
    print("The habit groups found in {}".format(home))
    for d, d_groups in groups.items():
        print(d + " " + str(len(d_groups)))


The habit groups found in HomeA/2016
Range 11
Microwave 10
LivingLights 113
HomeOffice 35
The habit groups found in HomeB/2016
LivingLights 71
Microwave 15
The habit groups found in HomeC/2016
HomeOffice 64
Microwave 18
LivingLights 76
The habit groups found in HomeD/2016
Microwave 10
LivingLights 38
WashingMachine 13
Range 26
The habit groups found in HomeF/2016
Microwave 40
WashingMachine 20
The habit groups found in HomeG/2016
LivingLights 32
HomeOffice 57
Range 22
Microwave 18


In [17]:
c_detector = ConflictDetector(ctx_info, capacity)
final_conflicts = c_detector.predict_conflict_scenarios(habit_groups)
print({x:len(final_conflicts[x]) for x in final_conflicts})

{'Range': 80, 'Microwave': 55, 'LivingLights': 0, 'HomeOffice': 90, 'WashingMachine': 41}


In [27]:
import plotly.express as px
probs = [(x["box"],x["prob"]) for x in final_conflicts["Microwave"]]
probs_i = [(x["prob"], i) for i, x in enumerate(final_conflicts["Microwave"])]
print(sorted(probs_i))
# fig = px.histogram(probs)
# fig.show()

[([42.0, 1.0, 43.0, 1.0], 0.0009459223629809006), ([40.0, 4.0, 43.0, 4.0], 0.0011104432644348586), ([30.0, 0.0, 36.0, 2.0], 0.011393596494700672), ([41.0, 4.0, 52.0, 6.0], 0.2750096602495074), ([21.0, 4.0, 21.0, 5.0], 0.018494330949880382), ([27.0, 4.0, 27.0, 6.0], 0.006913804204249058), ([25.0, 5.0, 25.0, 5.0], 0.008467075238252922), ([60.0, 0.0, 62.0, 5.0], 0.0011201306042514028), ([27.0, 6.0, 32.0, 6.0], 0.02841608385366986), ([60.0, 5.0, 62.0, 6.0], 0.004920179956580298), ([53.0, 6.0, 55.0, 6.0], 0.011741078081800603), ([60.0, 1.0, 61.0, 6.0], 0.017396163526161933), ([60.0, 0.0, 71.0, 3.0], 0.0004798531987394517), ([25.0, 6.0, 25.0, 6.0], 0.01870518500576504), ([53.0, 0.0, 59.0, 3.0], 0.058668543378338475), ([26.0, 1.0, 27.0, 5.0], 0.05335996130751907), ([27.0, 0.0, 36.0, 0.0], 0.03457074379401403), ([51.0, 0.0, 51.0, 5.0], 0.5093440677430189), ([51.0, 3.0, 52.0, 3.0], 0.006114286663210115), ([22.0, 4.0, 23.0, 5.0], 0.020105027346685508), ([44.0, 0.0, 54.0, 0.0], 0.1228517411079107

In [19]:
import copy
# Make ground truth:
ratio = 0.3
device_events = {}
for p in test_projects:
    ctx_evts, device_evts = load_processed(p)
    device_events[p] = device_evts

gtconflict_cfg = {
    "context_info": ctx_info,
    "capacity": capacity
}

conflict_finder = GtConflictFinder(gtconflict_cfg)
conflicts = conflict_finder.get_Gt_conflict(ctx_evts, device_events)
print(len(conflicts))

1663


In [20]:
import plotly.express as px

TEST_RATIO = 0.3
conflict_time = [x['cur_time'] for x in conflicts]
end_time = max(conflict_time)
start_time = min(conflict_time)
print(start_time)
print(end_time)
total_time_range = end_time - start_time
test_start = end_time - total_time_range * TEST_RATIO
print(test_start)

conflict_device = {
    d:[]
    for d in capacity
}
for c in conflicts:
    if c["cur_time"] > test_start:
        conflict_device[c["device"]].append(c)
print({d:len(c) for d,c in conflict_device.items()})


2016-01-01 15:07:00
2016-12-29 16:48:00
2016-09-11 18:41:42
{'Range': 0, 'Microwave': 197, 'LivingLights': 0, 'HomeOffice': 319, 'WashingMachine': 13}


In [26]:

missed_gt_conflicts = {d:[] for d in conflict_device}
hit_gt_conflicts = {d:{} for d in conflict_device}
for d in final_conflicts:
    hit_gt_conflicts[d] = {i:0 for i in range(len(final_conflicts[d]))}
for d, conflicts in conflict_device.items():
    for c in conflicts:
        coor = ctx_info.get_coor_by_ctx(c["ctx"])
        flag = False
        for idx,c_predict in enumerate(final_conflicts[d]):
            if does_contain_point(c_predict["box"], coor):
                flag = True
                hit_gt_conflicts[d][idx] += 1
                break
        if not flag:
            missed_gt_conflicts[d].append(c)
print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
hit_count = {d:[0,0] for d in hit_gt_conflicts}
for d,hit_c in hit_gt_conflicts.items():
    for i,h in hit_c.items():
        if h == 0:
            hit_count[d][0] += 1
        else:
            hit_count[d][1] += 1
print(hit_gt_conflicts["Microwave"])
print(final_conflicts["Microwave"][14])
print(hit_count)

{'Range': 0, 'Microwave': 1, 'LivingLights': 0, 'HomeOffice': 6, 'WashingMachine': 0}
{0: 0, 1: 0, 2: 12, 3: 15, 4: 0, 5: 1, 6: 0, 7: 4, 8: 1, 9: 1, 10: 8, 11: 0, 12: 2, 13: 0, 14: 33, 15: 1, 16: 0, 17: 7, 18: 0, 19: 3, 20: 4, 21: 0, 22: 0, 23: 0, 24: 5, 25: 1, 26: 0, 27: 8, 28: 1, 29: 9, 30: 2, 31: 0, 32: 0, 33: 9, 34: 0, 35: 0, 36: 2, 37: 0, 38: 0, 39: 2, 40: 7, 41: 2, 42: 5, 43: 19, 44: 0, 45: 0, 46: 0, 47: 18, 48: 1, 49: 1, 50: 7, 51: 2, 52: 0, 53: 2, 54: 1}
{'users': {(4, 37), (4, 4), (3, 9), (4, 12), (5, 3)}, 'box': [53.0, 0.0, 59.0, 3.0], 'prob': 0.058668543378338475, 'type': 'Capacity'}
{'Range': [80, 0], 'Microwave': [22, 33], 'LivingLights': [0, 0], 'HomeOffice': [35, 55], 'WashingMachine': [37, 4]}


In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

draw_device = "HomeOffice"
c_point = [[],[]]
missed_point = [[], []]
for c in conflict_device[draw_device]:
    coor = ctx_info.get_coor_by_ctx(c["ctx"])
    c_point[0].append(coor[0])
    c_point[1].append(coor[1])
for c in missed_gt_conflicts[draw_device]:
    coor = ctx_info.get_coor_by_ctx(c["ctx"])
    missed_point[0].append(coor[0])
    missed_point[1].append(coor[1])
fig = go.Figure(data=go.Scatter(x=c_point[0], y=c_point[1], mode="markers"))
fig.add_trace(go.Scatter(x=missed_point[0], y=missed_point[1], mode="markers", marker=dict(color="red")))

for c in final_conflicts[draw_device]:
    fig.add_shape(type="rect",
        x0=c["box"][0], y0=c["box"][1], x1=c["box"][2], y1=c["box"][3],
        line=dict(color="RoyalBlue"),
    )
fig.show()


In [29]:
hit_miss_rate = [[],[]]
for prob_threshold in np.arange(0.0, 0.5, 0.001):
    test_conflict = {
        d: [x for x in final_conflicts[d] if x["prob"] >= prob_threshold]
        for d in final_conflicts
    }
    missed_gt_conflicts = {d:[] for d in conflict_device}
    hit_gt_conflicts = {d:{} for d in conflict_device}
    for d in test_conflict:
        hit_gt_conflicts[d] = {i:0 for i in range(len(test_conflict[d]))}
    for d, conflicts in conflict_device.items():
        for c in conflicts:
            coor = ctx_info.get_coor_by_ctx(c["ctx"])
            flag = False
            for idx,c_predict in enumerate(test_conflict[d]):
                if does_contain_point(c_predict["box"], coor):
                    flag = True
                    hit_gt_conflicts[d][idx] += 1
                    break
            if not flag:
                missed_gt_conflicts[d].append(c)
    all_missed_gt_conflicts = sum([len(x) for d,x in missed_gt_conflicts.items()])
    all_gt_conflicts = sum([len(x) for d,x in conflict_device.items()])
    hit_miss_rate[0].append(1.0 - float(all_missed_gt_conflicts) / all_gt_conflicts)
    # print({d:len(missed_gt_conflicts[d]) for d in missed_gt_conflicts})
    all_pred_conflicts = 0
    all_missed_pred_conflicts = 0
    hit_count = {d:[0,0] for d in hit_gt_conflicts}
    for d,hit_c in hit_gt_conflicts.items():
        for i,h in hit_c.items():
            all_pred_conflicts += 1
            if h == 0:
                all_missed_pred_conflicts += 1
                hit_count[d][0] += 1
            else:
                hit_count[d][1] += 1
    hit_miss_rate[1].append(1.0 - float(all_missed_pred_conflicts) / all_pred_conflicts)
    # print(hit_count)

# print(hit_miss_rate)

In [24]:
print(len(hit_miss_rate[0]))
fig = go.Figure(data=go.Scatter(x=hit_miss_rate[0], y=hit_miss_rate[1], mode="lines+markers"))
fig.show()




500


In [11]:
from rtree import index


draw = {
    "HomeA/2016": ["Range"],
    "HomeG/2016": ["Range"],
}
fig = make_subplots(rows=sum([len(draw[x]) for x in draw]), cols=1)
p = index.Property()
p.dimension = len(ctx_info.get_all_ctx_ordered())
row_id = 1
for home in draw:
    for d in draw[home]:
        r_tree = index.Index(properties=p)
        for i,x in enumerate(habit_groups[home][d]):
            bound = x["box"][0] + x["box"][1] 
            r_tree.insert(id=i, coordinates=bound, obj=sum(x["dis"]))
        on_rate_group = np.full(ctx_info.get_ctx_space_shape(), -0.1)

        for i in range(ctx_info.get_ctx_space_shape()[0]):
            for j in range(ctx_info.get_ctx_space_shape()[1]):
                result = list(r_tree.intersection([i,j,i,j], objects=True))

                on_rate_group[i,j] = result[0].object
        fig.append_trace(
            go.Heatmap(z=np.transpose(on_rate_group)),
            row_id, 1
        )
        row_id += 1
fig.show()

AttributeError: 'str' object has no attribute 'type'

In [3]:
from ConflictDetector import device_capacity_conflict
print(ctx_info.get_space_area())
print(ctx_info.get_ctx_space_shape())
mins = ctx_info.get_coor_by_ctx({TIME_CTX:0, WEEKDAY_CTX:0})
maxs = ctx_info.get_coor_by_ctx({TIME_CTX:24*60, WEEKDAY_CTX:6})
print(compute_area([0,0, 72,6]))

432
(73, 7)
432
